In [52]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import r2_score

In [53]:
df = pd.read_parquet("/Users/ayush/Documents/University/Year 03/Sem 01/DATA3888/Optiver-07/Data/FE30Stocks.parquet")

In [54]:
print("Initial shape:", df.shape)
assert df.isnull().sum().sum() == 0, "DataFrame contains NaNs. Clean before proceeding."

Initial shape: (44391124, 26)


In [55]:
exclude_cols = ["rv_future", "time_id", "seconds_in_bucket", "stock_id"]
X_cols = [col for col in df.columns if col not in exclude_cols]
X = df[X_cols].values.astype(np.float32)
y = df["rv_future"].values.astype(np.float32)

In [56]:
rolling_vol = df["rolling_vol_30"].values.astype(np.float32)
weights = 1.0 / (rolling_vol ** 2 + 1e-6)

In [57]:
X = sm.add_constant(X)

In [58]:
model = sm.WLS(y, X, weights=weights)
results = model.fit()
print("β̂ shape :", results.params.shape)
print("β̂ head  :", results.params[:5])

β̂ shape : (23,)
β̂ head  : [ 1.02730659e-03 -5.71128329e-04  5.54984730e-01 -5.02026722e-07
  3.88579609e-07]


In [ ]:
epsilon = 1e-6
y_pred = np.clip(y_pred, epsilon, None)
y_true = np.clip(y, epsilon, None)
ratio = y_true / y_pred
qlike = np.mean(ratio - np.log(ratio) - 1.0)
print("QLIKE    :", qlike)
r2 = r2_score(y, y_pred)
print("R²       :", r2)

QLIKE    : 0.2808245775355561
R²       : 0.11170312960389006
